In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
from alpenglow.evaluation import DcgScore

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

In [ ]:
import sys
from datawand.parametrization import ParamHelper
ph = ParamHelper('..', 'LinkPrediction', sys.argv)

In [ ]:
links_df = pd.read_csv("/mnt/idms/fberes/data/bitcoin_ln_research/link_prediction/data/links_df_20.csv")

In [ ]:
links_df.head()

# Parameters

In [ ]:
K = 20#None#30#ph.get("top_first_days")
top_k = 100

In [ ]:
if K == None:
    delta_t = 86400*7
else:
    delta_t = 86400#

In [ ]:
models = [
    "onmf_dim10_lr0.140_nr100",
    "bomf_dim10_lr0.140_nr100",
    "offmf_dim10_lr0.050_nr100",
    "pop",
    "time_pop"
]

# Rankings

In [ ]:
ranking_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/link_prediction/rankings/topk%i_exkTrue_%s/" % (top_k,str(K))

In [ ]:
rankings = [pd.read_csv("%s/%s.csv" % (ranking_dir,m)) for m in models]

In [ ]:
[len(df) for df in rankings]

In [ ]:
rankings[0]['time'].min()

In [ ]:
START_TIME = 1548982800 # (GMT): Friday, February 1, 2019 1:00:00 AM

In [ ]:
def get_timeframe(df, delta_t, min_time=1548982800):
    df["timeframe"] = df["time"].apply(lambda x: max(0,(x-min_time)//delta_t))

for i in range(len(rankings)):
    get_timeframe(rankings[i], delta_t)

In [ ]:
rankings[4].isnull().sum()

# Results

### a.) average performance (online DCG)

The average performance for the offline batch model is confusing (it is only bad on the first day)

In [ ]:
def show_mean_dcg(with_first_day=True):
    if with_first_day:
        mean_dcgs = [df["dcg"].mean() for df in rankings]
        df = rankings[0]
        print(len(df))
    else:
        mean_dcgs = [df[df["timeframe"]>0]["dcg"].mean() for df in rankings]
        df = rankings[0]
        print(len(df[df["timeframe"]>0]))
    return pd.DataFrame(list(zip(models, mean_dcgs)), columns=["model","dcg"]).sort_values("dcg", ascending=False).reset_index(drop=True)

#### Global mean performance

In [ ]:
show_mean_dcg(True)

#### Mean performance without first day

In [ ]:
show_mean_dcg(False)

**Exclude known: False**
0 	online 	0.139660
1 	batch+online 	0.131745
2 	pop+time 	0.124183
3 	pop 	0.077110
4 	batch 	0.064587

**Exclude known: True - Miért teljesen uaz?**
0 	online 	0.139660
1 	batch+online 	0.131745
2 	pop+time 	0.124183
3 	pop 	0.077110
4 	batch 	0.064587

### b.) Performance over time

In [ ]:
for idx, ranking in enumerate(rankings):
    averages = ranking.groupby("timeframe")["dcg"].mean()
    plt.plot(averages, label=models[idx])
plt.legend()

### c.) Number of records over time

In [ ]:
cnt = rankings[0].groupby("timeframe")["dcg"].count()
plt.plot(cnt)

# Simulation based results

experiment_id = "200000sat_k10000_aNone_e0.05_dropTrue-onmf_dim10_lr0.140_nr100"

In [ ]:
def load_link_sim_experiment(model_dir):
    model_id = model_dir.split("/")[-2]
    print(model_id)
    model_files = os.listdir(model_dir)
    chunks = [pd.read_csv("%s/%s" % (model_dir, f)) for f in model_files if "snapshot" in f]
    concatenated = pd.concat(chunks)
    print(len(model_files), len(concatenated))
    get_timeframe(concatenated, delta_t)
    #print(concatenated.isnull().sum() / len(concatenated))
    #print(concatenated.head())
    for rank_col in ["rank1","rank2","rank3","rank4"]:
        print(rank_col)
        concatenated["dcg_"+rank_col] = DcgScore(concatenated.rename({rank_col:"rank"}, axis=1))
        print(concatenated[rank_col].mean())
    return concatenated
    #concatenated[model_id] = DcgScore(concatenated)
    #print(concatenated[model_id].mean())
    #concatenated['base_dcg'] = DcgScore(concatenated.drop("rank",axis=1).rename({"base":"rank"}, axis=1))
    #print(concatenated["base_dcg"].mean())
    #return concatenated.drop("base_dcg", axis=1)

experiments = [
    "200000sat_k10000_aNone_e0.05_dropTrue-onmf_dim10_lr0.140_nr100",
    "200000sat_k10000_aNone_e0.05_dropFalse-onmf_dim10_lr0.140_nr100",
    "200000sat_k10000_a2.0_e0.05_dropTrue-onmf_dim10_lr0.140_nr100"
]

experiments = [
"200000sat_k10000_aNone_e0.05_dropTrue-onmf_dim10_lr0.140_nr100",
#"100000sat_k10000_aNone_e0.05_dropTrue-onmf_dim10_lr0.140_nr100",
#"200000sat_k10000_a2.0_e0.05_dropTrue-onmf_dim10_lr0.140_nr100",
#"200000sat_k10000_aNone_e0.05_dropFalse-onmf_dim10_lr0.140_nr100",
#"200000sat_k20000_aNone_e0.05_dropTrue-onmf_dim10_lr0.140_nr100",
#"200000sat_k5000_aNone_e0.05_dropTrue-onmf_dim10_lr0.140_nr100",
#"300000sat_k10000_aNone_e0.05_dropTrue-onmf_dim10_lr0.140_nr100",
"500000sat_k10000_aNone_e0.05_dropTrue-onmf_dim10_lr0.140_nr100",
"200000sat_k10000_aNone_e0.05_dropTrue-time_pop",
"500000sat_k10000_aNone_e0.05_dropTrue-time_pop",
"sp_200000sat_k10000_aNone_e0.05_dropTrue-onmf_dim10_lr0.140_nr100",
"sp_200000sat_k10000_aNone_e0.05_dropTrue-time_pop",
"sp_500000sat_k10000_aNone_e0.05_dropTrue-onmf_dim10_lr0.140_nr100",
"sp_500000sat_k10000_aNone_e0.05_dropTrue-time_pop",
"slink_200000sat_k10000_aNone_e0.05_dropTrue-onmf_dim10_lr0.140_nr100",
"slink_200000sat_k10000_aNone_e0.05_dropTrue-time_pop",
"slink_500000sat_k10000_aNone_e0.05_dropTrue-onmf_dim10_lr0.140_nr100",
"slink_500000sat_k10000_aNone_e0.05_dropTrue-time_pop",
]

In [ ]:
experiments = ["50000sat_k6000_aNone_e0.05_dropTrue-time_pop"]#["trial"]

In [ ]:
simulation_results = [load_link_sim_experiment("%s/%s/" % (ranking_dir, experiment_id)) for experiment_id in experiments]

In [ ]:
sim_preds = simulation_results[0]

In [ ]:
for idx, model in enumerate(experiments[1:]):
    sim_preds = sim_preds.merge(simulation_results[idx+1][["record_id",model]], on="record_id", how="left")

In [ ]:
sim_preds.head()

### Joining baselines with simulation results

In [ ]:
for idx, model in enumerate(models):
    sim_preds = sim_preds.merge(rankings[idx][["id","dcg"]].rename({"id":"record_id","dcg":model}, axis=1), on="record_id", how="left")

sim_preds[experiments+models].mean().sort_values(ascending=False)

In [ ]:
sim_preds[models+["dcg_rank1","dcg_rank2","dcg_rank3","dcg_rank4"]].mean().sort_values(ascending=False)

# Rank combination

In [ ]:
onmf_preds = pd.read_csv("%s/preds_onmf_dim10_lr0.140_nr100.csv" % ranking_dir)
onmf_preds = onmf_preds.drop("dcg", axis=1)
onmf_preds.shape

In [ ]:
tpop_preds = pd.read_csv("%s/preds_time_pop.csv" % ranking_dir)
tpop_preds = tpop_preds.drop("dcg", axis=1)
tpop_preds.shape

In [ ]:
sim_model = experiments[0]
sim_preds_parts = [pd.read_csv("%s/%s/preds_%i.csv" % (ranking_dir,sim_model,i)) for i in range(19)]
sim_preds = pd.concat(sim_preds_parts, ignore_index=True)
sim_preds.shape

## TODO: rájönni, hogy mi okozza a degree based baseline visszaesését! Elvileg hasonló csúcsokat jósol a szimuláció is...

## TODO: csak a tiszta predikciókat kéne kombinálni (ui. ritka event az amikor bevétel szerint optimalizálna valaki)

node_names = pd.read_csv("/mnt/idms/fberes/data/bitcoin_ln_research/node_names.csv")

only_sim_preds = sim_preds[sim_preds["prediction"]>0]

only_cap_preds = sim_preds[sim_preds["prediction"]==0]

def show_avg_rank(df):
    item_counts = df["item"].value_counts()
    most_pop = list(item_counts[item_counts > 100].index)
    item_avg_ranks = df[df["item"].isin(most_pop)].groupby("item")["rank"].mean().sort_values().reset_index()
    return item_avg_ranks.merge(node_names[["pub_key","name","cap_ratio"]], left_on="item", right_on="pub_key", how="left")

only_sim_ranks = show_avg_rank(only_sim_preds)

only_cap_ranks = show_avg_rank(only_cap_preds)

only_sim_ranks.head(10)

only_cap_ranks.head(10)

### filter for common records

In [ ]:
common_record_ids = set(sim_preds["record_id"])
len(common_record_ids)

In [ ]:
onmf_preds = onmf_preds[onmf_preds["record_id"].isin(common_record_ids)]
onmf_preds.shape

In [ ]:
tpop_preds = tpop_preds[tpop_preds["record_id"].isin(common_record_ids)]
tpop_preds.shape

### extract real targets

In [ ]:
real_targets = dict(links_df.loc[common_record_ids]["item"])

In [ ]:
len(real_targets)

### recoding keys to ids

In [ ]:
node_2_id = dict(zip(links_df["src"],links_df["user"]))
node_2_id.update(dict(zip(links_df["trg"],links_df["item"])))

In [ ]:
sim_preds["user"] = sim_preds["user"].apply(lambda x: node_2_id[x])
sim_preds["item"] = sim_preds["item"].apply(lambda x: node_2_id[x])

In [ ]:
sim_preds.head()

### combination

In [ ]:
import numpy as np
from tqdm import tqdm

def performance(pred_df,real_targets):
    """real_targets contains true items for only the related sessions"""
    df = pred_df.copy()
    df["true_item"] = df["record_id"].apply(lambda x: real_targets[x])
    hits = df[df["true_item"]==df["item"]]
    hits["dcg"] = 1.0 / (np.log2(hits["rank"]+1.0))
    return hits["rec_rank"].sum() / len(real_targets), hits["dcg"].sum() / len(real_targets)

def combine_ranks(preds_1, preds_2, alpha, k):
    cols = ["record_id","item","rec_rank"]
    p1 = preds_1[cols].copy()
    p2 = preds_2[cols].copy()
    p1["rec_rank"] = p1["rec_rank"]*alpha
    p2["rec_rank"] = p2["rec_rank"]*(1.0-alpha)
    combined = pd.concat([p1,p2], ignore_index=True)
    combined = combined.groupby(["record_id","item"])["rec_rank"].sum().reset_index()
    combined = combined[combined["rec_rank"]>0]
    combined["rank"] = 1.0 / combined["rec_rank"]
    combined["dcg"] = DcgScore(combined)
    # TODO: filter for top_k recommendation for each event???
    return combined

def combine_models(model1, model2, alphas):
    mrrs, mdcgs = [], []
    #for a in tqdm(alphas):
    for a in alphas:
        combi = combine_ranks(model1, model2, a, top_k)
        mrr, mdcg = performance(combi, real_targets)
        mrrs.append(mrr)
        mdcgs.append(mdcg)
        print(len(combi)/len(model2))
    print(mrrs)
    print(mdcgs)
    #plt.plot(alphas,mrrs,'bx',label="mrr")
    plt.plot(alphas,mdcgs,'rx',label="mdcg")
    plt.legend()

In [ ]:
onmf_preds["rec_rank"] = 1.0 / onmf_preds["rank"]
sim_preds["rec_rank"] = 1.0 / sim_preds["rank"]
tpop_preds["rec_rank"] = 1.0 / tpop_preds["rank"]

In [ ]:
only_sim_preds = sim_preds[sim_preds["prediction"]>0]

In [ ]:
len(only_sim_preds), len(sim_preds)

In [ ]:
print(performance(onmf_preds, real_targets))
print(performance(tpop_preds, real_targets))
print(performance(sim_preds, real_targets))
print(performance(only_sim_preds, real_targets))

In [ ]:
#alphas = np.arange(0,1.1,0.1)
#alphas = [0.0,0.025,0.05,0.075,0.1,0.2,0.3,0.5,1.0]
alphas = [0.0,0.01,0.02,0.03,0.04,0.05,0.1,0.5,1.0]

In [ ]:
combine_models(tpop_preds, onmf_preds, alphas)

In [ ]:
combine_models(sim_preds, onmf_preds, alphas)

In [ ]:
combine_models(only_sim_preds, onmf_preds, alphas)

# Betweeness baseline

from link_pred_simulator import load_graph_snapshots, process_links_for_simulator

snapshots, time_boundaries = load_graph_snapshots("/mnt/idms/fberes/data/bitcoin_ln_research/directed_graphs/directed_temporal_multi_edges_1days.csv")

links_for_sim = process_links_for_simulator(links_df, None, time_boundaries, only_eval=False, verbose=True)

links_for_sim["eval"].value_counts()

from tqdm import tqdm

def calculate_rank(scores_df, true_target, top_k, seen_nodes=None):
        # scores_df is preordered
        if seen_nodes == None:
            ordered_list = list(scores_df["index"])[:top_k]
        else:
            ordered_list = list(scores_df[~scores_df["index"].isin(seen_nodes)]["index"])[:top_k]
        return ordered_list.index(true_target)+1.0 if true_target in ordered_list else None

def extract_central_nodes(file_path, metric):
    scores_df = pd.read_csv(file_path, usecols=["index","betw"])
    return scores_df.sort_values(metric, ascending=False)[["index",metric]]
    
class BetweenessModel():
    def __init__(self, centrality_dir="/mnt/idms/fberes/data/bitcoin_ln_research/centrality_scores", weight=None):
        self.metric = "betw"
        self.weight = weight
        self.centrality_dir = centrality_dir
        self.top_k_preds = {}
        
    def run(self, links, k, exclude_known=True):
        self.graph = {}
        max_snap_id = links["snapshot"].max()
        for snap_id in range(max_snap_id+1):
            f_path = "%s/scores_%s_%i.csv" % (self.centrality_dir, self.weight, snap_id)
            self.top_k_preds[snap_id] = extract_central_nodes(f_path, self.metric)
        ranks = []
        indices = links.index
        for idx in tqdm(indices):
            row = links.loc[idx]
            snap_id, src, trg, eval_ = row["snapshot"], row["src"], row["trg"], row["eval"]
            if not src in self.graph:
                self.graph[src] = set()
            if eval_:
                seen_nodes = self.graph[src] if exclude_known else None
                ranks.append(calculate_rank(self.top_k_preds[snap_id], trg, k, seen_nodes))
            else:
                ranks.append(None)
            self.graph[src].add(trg)
        rankings = links.copy()
        rankings["rank"] = ranks
        return rankings[rankings["eval"]==1]

# How can it be the same DCG???

bm = BetweenessModel()

bm_rankings = bm.run(links_for_sim, 20, exclude_known=True)

bm_rankings["dcg"] = DcgScore(bm_rankings)

bm_rankings["dcg"].mean()

bm_rankings_f = bm.run(links_for_sim, 20, exclude_known=False)

bm_rankings["dcg"] = DcgScore(bm_rankings)

bm_rankings["dcg"].mean()